In [1]:
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
from google.colab import files
myfile = files.upload()

Saving anime_with_synopsis.csv to anime_with_synopsis.csv


In [9]:
anime_with_synopsis = pd.read_csv("/content/anime_with_synopsis.csv")

In [10]:
anime_with_synopsis.head(10)

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...
5,15,Eyeshield 21,7.95,"Action, Sports, Comedy, Shounen",Sena is like any other shy kid starting high s...
6,16,Hachimitsu to Clover,8.06,"Comedy, Drama, Josei, Romance, Slice of Life","Yuuta Takemoto, a sophomore at an arts college..."
7,17,Hungry Heart: Wild Striker,7.59,"Slice of Life, Comedy, Sports, Shounen",Kyosuke Kano has lived under the shadow of his...
8,18,Initial D Fourth Stage,8.15,"Action, Cars, Sports, Drama, Seinen",Takumi Fujiwara finally joins Ryousuke and Kei...
9,19,Monster,8.76,"Drama, Horror, Mystery, Police, Psychological,...","Dr. Kenzou Tenma, an elite neurosurgeon recent..."


In [11]:
anime_with_synopsis = anime_with_synopsis[['MAL_ID', 'Name', 'sypnopsis']]

In [13]:
anime_with_synopsis = anime_with_synopsis.drop_duplicates()
anime_with_synopsis = anime_with_synopsis.dropna()
anime_with_synopsis['sypnopsis'] = anime_with_synopsis['sypnopsis'].fillna('')
len(anime_with_synopsis)

16206

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')         
tfidf_matrix = tfidf.fit_transform(anime_with_synopsis['sypnopsis'])

In [15]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)                     
cosine_sim = pd.DataFrame(cosine_sim, index=anime_with_synopsis.index, columns=anime_with_synopsis.index) 

In [16]:
indices = pd.Series(anime_with_synopsis.index, index=anime_with_synopsis['Name'])

In [17]:
def content_recommender(name, n_of_recomm):
    idx = indices[name]
    sim_scores = cosine_sim[idx]
    sim_scores = sim_scores.sort_values(ascending=False)[1:n_of_recomm+1]
    return anime_with_synopsis.loc[sim_scores.index]['Name']

In [19]:
content_recommender("Cowboy Bebop", 3)

1        Cowboy Bebop: Tengoku no Tobira
3149                Ginga Senpuu Braiger
6703    Cowboy Bebop: Ein no Natsuyasumi
Name: Name, dtype: object